## Different perspective
1. PCA
2. manual
   - encode trustLevel
   - delete 'valuePerSecond', 'scannedLineItemsPerSecond', 'lineItemVoidsPerPosition'
   - generate no. item = totalScanTimeInSeconds * scannedLineItemsPerSecond
3. automatic generation

## Code for test
prepare different input dataset and test at 10-fold stratified cross validation set
1. train_data (raw data)
2. X_train_norm_enc data (normalized and encode)
3. X_train_manual (manual)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
from classifiers import *
from sklearn.metrics import confusion_matrix

## Original

In [2]:
train_data = pd.read_csv('data/train.csv', sep = '|')
test_data = pd.read_csv('data/test.csv', sep = '|')
print(f'Train set has {train_data.shape[0]} entries and {train_data.shape[1]} features')
print(f'Test set has {test_data.shape[0]} entries and {test_data.shape[1]} features')

y = train_data['fraud']
X = train_data.drop(columns=['fraud']).astype(float)
fit_minmax = MinMaxScaler()
X_encode = pd.get_dummies(X, columns=['trustLevel'], prefix='trustLevel')
X_train_norm_enc = pd.DataFrame(fit_minmax.fit_transform(X_encode), columns=X_encode.columns, index=X_encode.index)
print(X_train_norm_enc.shape)
X_test_encode = pd.get_dummies(test_data, columns=['trustLevel'], prefix='trustLevel')
X_test_norm_enc = pd.DataFrame(fit_minmax.transform(X_test_encode), columns=X_test_encode.columns, index=X_test_encode.index)



Train set has 1879 entries and 10 features
Test set has 498121 entries and 9 features
(1879, 14)


/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


## Manual delete and create

In [3]:
## delete correlate features
X_manual = X.assign(no_item = X.totalScanTimeInSeconds* X.scannedLineItemsPerSecond)\
                     .drop(columns=['valuePerSecond', 'scannedLineItemsPerSecond', 'lineItemVoidsPerPosition'])
fit_minmax = MinMaxScaler()
X_manual_encode = pd.get_dummies(X_manual, columns=['trustLevel'], prefix='trustLevel')
X_train_manual = pd.DataFrame(fit_minmax.fit_transform(X_manual_encode), columns=X_manual_encode.columns, index=X_manual_encode.index)
print(X_train_manual.shape)

X_test = test_data.assign(no_item = test_data.totalScanTimeInSeconds* test_data.scannedLineItemsPerSecond)\
                     .drop(columns=['valuePerSecond', 'scannedLineItemsPerSecond', 'lineItemVoidsPerPosition'])
X_test_encode = pd.get_dummies(X_test, columns=['trustLevel'], prefix='trustLevel')
X_test_manual = pd.DataFrame(fit_minmax.transform(X_test_encode), columns=X_test_encode.columns, index=X_test_encode.index)
print(X_test_manual.shape)



/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


(1879, 12)
(498121, 12)


In [4]:
def code_for_test(X,y):
    """
    test code with 10-fold stratified cross validation
    parameters
    X: trainset features after generation
    y: trainset y label
    """
    evaluate_classification(X, y)

In [5]:
code_for_test(X_train_norm_enc, y)

K Nerest Neighbors: test core = -710 
Linear SVM: test core = -520 
RBF SVM: test core = -610 
Logistic Regression: test core = -630 
Decision Tree: test core = -785 
Neural Net: test core = -505 
Random Forest: test core = -155 
AdaBoost: test core = -65 
XGBoost: test core = -60 


In [6]:
code_for_test(X_train_manual, y)

K Nerest Neighbors: test core = -65 
Linear SVM: test core = -520 
RBF SVM: test core = 225 
Logistic Regression: test core = 205 
Decision Tree: test core = -260 
Neural Net: test core = 170 
Random Forest: test core = 175 
AdaBoost: test core = 135 
XGBoost: test core = 80 
